
## <font color=red> You should not import any new libraries. Your code should run with python=3.x</font>
## <font color=red> Please don't rename this .ipynb file.</font><br>
- Your solutions will be auto-graded. Hence we request you to follow the instructions.
- Modify the code only between 
```
## TODO
## END TODO
```
- In addition to above changes, you can play with arguments to the functions for generating plots
- We will run the auto grading scripts with private test data

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy.linalg import inv  # use this function to invert matrices

## Please make sure that your code works with loading data from relative path only

i.e. ```pd.read_csv('./data/single_var_new.csv')``` should not throw an error when we run the auto-grading scripts

In [ ]:
data_single = pd.read_csv('./data/single_var.csv')
X_single = np.array(data_single['x_gt'])
Y_single = np.array(data_single['y_gt'])

data_multi = pd.read_csv('./data/multi_var.csv')
cols = [f"x_gt_{idx}" for idx in range(1, 6)]
X_multi = np.array(data_multi[cols])
Y_multi = np.array(data_multi['y_gt'])

## Plot Graphs

- This function plots the ground truth curve in <font color=green>green</font> and the predicted function in <font color=red>red</font>

In [ ]:
def plot_curves(w, b, x, y):
    '''
    Plots the curves for groud truth function and the fitted function

    Args:
    w - list of parameters
    b - float
    x - data features
    y - output value
    '''

    assert type(w) == type([0]), f'Please pass a list of parameters to plot_curves and not {type(w)}'

    x_gt = np.linspace(-1, 2, 50)
    y_gt = 1 - 3 * x_gt - 2 * x_gt ** 2 + 2.5 * x_gt ** 3

    if len(w) == 1:
        y_fit = w * x_gt + b
    elif len(w) == 5:
        x_fit = x_gt
        for pow in range(2, 4):
            x_fit = np.vstack([x_fit, np.power(x_gt, pow)])

        x_fit = np.vstack([x_fit, np.sin(np.pi * 2 * x_gt)])
        x_fit = np.vstack([x_fit, np.cos(np.pi * x_gt)])
        y_fit = np.dot(w, x_fit) + b
    else:
        assert False, 'Pass a valid w'
    plt.plot(x_gt, y_gt, color="green", label='1 - 3 * x - 2 * x ** 2 + 2.5 * x ** 3')
    plt.plot(x_gt, y_fit, color='red', label="Fitted Function y = w.Tx + b")
    if len(x.shape) == 1:
        x_plot = np.vstack([x, np.ones(len(x))]).T
    else:
        x_plot = x
    plt.scatter(x_plot[:, 0], y)
    plt.legend()
    plt.title("OLS")
    plt.show()

## Split data into train/validation
- make sure that training and validation datasets are disjoint
- Split data into train and validation sets such that train contains floor(n_sampes * train_ratio) and test contains the remaining samples

In [ ]:
def split_data(X, Y, train_ratio=0.6):
    '''
    Split data into train and validation sets such that train
    contains floor(n_sampes * train_ratio) and test contains the remaining
    samples

    Args:
    X - numpy array of shape (n_samples, n_features)
    Y - numpy array of shape (n_samples, 1)
    train_ratio - fraction of samples to be used as training data

    Returns:
    X_train, Y_train, X_val, Y_val
    '''

    ## TODO
    n_samples = X.shape[0]
    train_size = int(n_samples * train_ratio)

    np.random.seed(0)
    indices = np.random.permutation(n_samples)
    X_train = X[indices[:train_size]]
    Y_train = Y[indices[:train_size]]
    X_val = X[indices[train_size:]]
    Y_val = Y[indices[train_size:]]

    ## END TODO

    return X_train, Y_train, X_val, Y_val

## mse for single variable regression

In [ ]:
def mse_single_var(X, Y, w, b):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 1)
    Y - numpy array of shape (n_samples, 1)
    w - a float
    b - a float
    '''

    ## TODO
    N = X.shape[0]
    mse = np.square(w*X + b - Y).mean() / N

    ## END TODO

    return mse

In [ ]:
def mse_multi_var(X, Y, w, b):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 5)
    Y - numpy array of shape (n_samples, 1)
    w - list of parameters
    b - a float
    '''

    ## TODO
    N = X.shape[0]
    mse = np.square(np.dot(X, w) + b - Y).mean() / N

    ## END TODO

    return mse

In [ ]:
def mse_regularized(X, Y, w, b, lamda):
    '''
    Compute mean squared error between predictions and true y values

    Args:
    X - numpy array of shape (n_samples, 5)
    Y - numpy array of shape (n_samples, 1)
    w - list of parameters
    b - a float
    lamda - a float
    '''

    ## TODO
    N = X.shape[0]
    mse = lamda * np.square(w).mean() + \
        np.square(np.dot(X, w) + b - Y).mean() / N

    ## END TODO

    return mse

# Single Variable Regression

In [ ]:
def singlevar_grad(X_train, Y_train, X_val, Y_val, epochs=100, lr=1e-3):
    '''
    Perform single variable least squares regression using gradient descent

    Args:
    X_train - numpy array of shape (n_samples_train, 1)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 1)
    Y_val - numpy array of shape (n_samples_val, 1)
    epochs - number of gradient descent steps
    lr - learning rate
    '''

    # Initialize the parameters with 0
    w = 0
    b = 0

    ## TODO
    N = X_train.shape[0]
    for i in range(epochs):
        grad_w = 2 / N * np.dot(w*X_train + b - Y_train, X_train)
        grad_b = 2 / N * np.sum(w*X_train + b - Y_train)
        w = w - lr * grad_w
        b = b - lr * grad_b

    ## END TODO

    mse_train = mse_single_var(X_train, Y_train, w, b)
    mse_val = mse_single_var(X_val, Y_val, w, b)
    print(f'Validation loss is {mse_val}')
    print(f'Training Loss loss is {mse_train}')
    plot_curves([w], b, X_train, Y_train)
    return w, b


def singlevar_closedform(X_train, Y_train, X_val, Y_val):
    '''
    Perform single variable least squares regression using closed form update 
    rules

    Args:
    X_train - numpy array of shape (n_samples_train, 1)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 1)
    Y_val - numpy array of shape (n_samples_val, 1)
    '''

    w = 0
    b = 0

    ## TODO
    N = X_train.shape[0]
    phi = np.array([[1]*N, X_train]).T
    b, w = inv(phi.T @ phi) @ phi.T @ Y_train

    ## END TODO

    mse_train = mse_single_var(X_train, Y_train, w, b)
    mse_val = mse_single_var(X_val, Y_val, w, b)
    print(f'Validation loss is {mse_val}')
    print(f'Training Loss loss is {mse_train}')
    plot_curves([w], b, X_train, Y_train)
    return w, b

In [ ]:
X_train, Y_train, X_val, Y_val = split_data(X_single, Y_single, train_ratio=0.6)

w, b = singlevar_grad(X_train, Y_train, X_val, Y_val, epochs=1000, lr=1e-2)
print(w, b)

print('----'*25)

w, b = singlevar_closedform(X_train, Y_train, X_val, Y_val)
print(w, b)

# Multi Variable Regression


In [ ]:
def multivar_grad(X_train, Y_train, X_val, Y_val, epochs=100, lr=1e-3):
    '''
    Perform multi variable least squares regression using gradient descent

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    epochs - number of gradient descent steps
    lr - learnig rate
    '''

    w = [0.0] * X_train.shape[1]
    b = 0

    ## TODO
    N = X_train.shape[0]
    for i in range(epochs):
        grad_w = 2 / N * np.dot(np.dot(X_train, w) + b - Y_train, X_train)
        grad_b = 2 / N * np.sum(np.dot(X_train, w) + b - Y_train)
        w = w - lr * grad_w
        b = b - lr * grad_b

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)
    return w, b


def multivar_closedform(X_train, Y_train, X_val, Y_val):
    '''
    Perform multi variable least squares regression using closed form 
    updates

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    '''

    w = [0.0] * X_train.shape[1]
    b = 0

    ## TODO
    N = X_train.shape[0]
    phi = np.hstack((X_train, np.ones((N, 1))))
    w_all = inv(phi.T @ phi) @ phi.T @ Y_train
    w = w_all[:-1]
    b = w_all[-1]

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)
    return w, b

In [ ]:
X_train, Y_train, X_test, Y_test = split_data(X_multi, Y_multi, train_ratio=0.6)

w, b = multivar_grad(X_train, Y_train, X_test, Y_test, epochs=1000, lr=1e-1)
print(w, b)

print('----' * 25)

w, b = multivar_closedform(X_train, Y_train, X_test, Y_test)
print(w, b)

# Regularization

In [ ]:
def multivar_reg_grad(X_train, Y_train, X_val, Y_val, epochs=100, lr=1e-3, lamda=1):
    '''
    Perform L2 regularized multi variable least squares regression using gradient descent

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    epochs - number of gradient descent steps
    lr - learnig rate
    lamda - regularization weight
    '''

    w = [0.0] * X_train.shape[1]
    b = 0

    ## TODO
    N = X_train.shape[0]
    for i in range(epochs):
        grad_w = 2 * lamda * np.array(w).T + \
            2 / N * np.dot(np.dot(X_train, w) + b - Y_train, X_train)
        grad_b = 2 / N * np.sum(np.dot(X_train, w) + b - Y_train)
        w = w - lr * grad_w
        b = b - lr * grad_b

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)
    return w, b


def multivar_reg_closedform(X_train, Y_train, X_val, Y_val, lamda=0.5):
    '''
    Perform L2 regularized multi variable least squares regression using 
    closed form update rules

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    lambda - regularization weight
    '''

    w = [0.0] * X_train.shape[1]
    b = 0

    ## TODO
    N = X_train.shape[0]
    phi = np.hstack((np.ones((N, 1)), X_train))
    weight = lamda * N * np.eye(X_train.shape[1] + 1)
    weight[0, 0] = 0
    w_all = inv(phi.T @ phi + weight) @ phi.T @ Y_train
    w = w_all[1:]
    b = w_all[0]

    ## END TODO

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')
    plot_curves(list(w), b, X_train, Y_train)
    return w, b

In [ ]:
X_train, Y_train, X_val, Y_val = split_data(X_multi, Y_multi, train_ratio=0.6)

w, b = multivar_reg_grad(X_train, Y_train, X_val, Y_val, epochs=1000, lr=1e-1, lamda=0.001)
print(w, b)

print('----' * 25)

w, b = multivar_reg_closedform(X_train, Y_train, X_val, Y_val, lamda=0.001)
print(w, b)

# Bayesian Linear Regression

As we studied in class, in Bayesian approach, we model the parameters of the model as a Random variable. 

We assume a (conjugate) prior distribution over the parameters ($p(w)$) and update  the posterior $p(w | D)$ based on the obervations.

In this assignment, we will assume that parameters are sampled from Gaussian distribution and try to learn the mean and variance of posterior.

Following the above discussion,

- let prior $p(w) = \mathcal{N}(\mu_0, \sum_0) = \mathcal{N}(0, 0.5)$
- We know that $p(w | \mathcal{D}) \propto p(w) P(\mathcal{D} | w)$
- Then, $p(w | \mathcal{D}) = \mathcal{N}(\mu_N, \sum_N)$
- And $p(\mathcal{D} | w) = \mathcal{N}(y| w^Tx, \sigma^2I)$

For this assignment, feel free to play with various values of $\sigma^2$ that gives you a better fit.

Finally the posterior parameters are given by, <br>
$\mu_N = \sum_N (\sum_0^{-1} \mu_0 + \frac{1}{\sigma^2}X^Ty)$ <br>
$\sum_N = (\sum_0^{-1} + \frac{1}{\sigma^2}X^TX)^{-1}$


### Learning Bias b
Eventhough the above formula shows how to compute $w$, we expect you to learn the bias $b$ also.

<font color='blue'> Hint: You can append a constant $1$ to all the input features $x$. Thus making the data $X \in \mathcal{R}^{d+1}$. </font>

In [ ]:
def bayesion_lr(X_train, Y_train, X_val, Y_val, sigma=0.01):
    '''
    Perform Bayesian Linear Regression

    Args:
    X_train - numpy array of shape (n_samples_train, 5)
    Y_train - numpy array of shape (n_samples_train, 1)
    X_val - numpy array of shape (n_samples_val, 5)
    Y_val - numpy array of shape (n_samples_val, 1)
    '''

    ## TODO
    N = X_train.shape[0]
    d = X_train.shape[1]

    X = np.hstack((np.ones((N, 1)), X_train))
    Y = np.array([Y_train]).T

    sigma_0 = 0.5 * np.eye(d+1)
    mu_0 = np.zeros((d+1, 1))

    sigma_N = inv(inv(sigma_0) + (X.T @ X / sigma**2))
    mu_N = sigma_N @ (inv(sigma_0) @ mu_0 + (X.T @ Y / sigma**2))

    w = list(mu_N[1:, 0])
    b = mu_N[0, 0]

    mse_train = mse_multi_var(X_train, Y_train, w, b)
    mse_val = mse_multi_var(X_val, Y_val, w, b)
    print(f'Validation loss if {mse_val}')
    print(f'Training Loss loss if {mse_train}')

    ## END TODO

    plot_curves(w, b, X_train, Y_train)

In [ ]:
X_train, Y_train, X_val, Y_val = split_data(X_multi, Y_multi, train_ratio=0.6)

bayesion_lr(X_train, Y_train, X_val, Y_val, sigma=0.1)

I have used ```@``` instead of ```np.matmul``` for matrix multiplication. ```@``` operator is available for Python 3.5+, please consider.